In [1]:
%load_ext autoreload
%autoreload
import tensorflow as tf
import numpy as np
import surreal, config
import functools, operator, copy
import tensorflow.contrib.slim as slim
from nets.resnet_v2 import resnet_v2_101
from nets.mobilenet import mobilenet_v2
from tensor_info import INPUT_TENSOR_INFO, OUTPUT_TENSOR_INFO
tf.reset_default_graph()
'''
for x in surreal.load():
    print(x)
    break
'''

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'\nfor x in surreal.load():\n    print(x)\n    break\n'

In [2]:
types = tuple(t['type'] for t in INPUT_TENSOR_INFO)

with tf.device('/cpu:0'):
    input_tensors = tf.data.Dataset.from_generator(surreal.load, types) \
                                   .batch(config.BATCH_SIZE) \
                                   .prefetch(config.PREFETCH_SIZE) \
                                   .make_one_shot_iterator() \
                                   .get_next()
    tensors = {}
    for tensor, info in zip(input_tensors, INPUT_TENSOR_INFO):
        tensor.set_shape(info['shape'])
        tensors[info['name']] = tensor

In [3]:
config.STRIDE = 16

MD_H = int(config.TAR_H/config.STRIDE)
MD_W = int(config.TAR_W/config.STRIDE)

DEPTH = [ti['shape'][-1] for ti in OUTPUT_TENSOR_INFO]
RESULT_SHAPE = (config.BATCH_SIZE, MD_H, MD_W, sum(DEPTH))
RESULT_SIZE = functools.reduce(operator.mul, RESULT_SHAPE[1:])
OUTPUT_SHAPE = (config.BATCH_SIZE, config.TAR_H, config.TAR_W, sum(DEPTH))
OUTPUT_SIZE = functools.reduce(operator.mul, OUTPUT_SHAPE[1:])


In [4]:
def bilinear(indices):
    oy = tf.clip_by_value(indices[1], 0, MD_H-1)
    ox = tf.clip_by_value(indices[2], 0, MD_W-1)
    iy = [tf.floor(oy), tf.ceil(oy)]
    ix = [tf.floor(ox), tf.ceil(ox)]
    idx_p = []
    for y_i in range(2):
        for x_i in range(2):
            indices[1] = iy[y_i]
            indices[2] = ix[x_i]
            idx = tf.cast(tf.stack(indices, axis=-1), tf.int32)
            p = (1 - tf.abs(iy[y_i] - oy)) * (1 - tf.abs(ix[x_i] - ox))
            idx_p.append((idx, p))
    return idx_p

def gather_bilinear(params, indices):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.gather_nd(params, idx)
        res.append(r * p)
    return tf.add_n(res)

def scatter_bilinear(params, indices, shape):
    idx_p = bilinear(indices)
    res = []
    for idx, p in idx_p:
        r = tf.scatter_nd(idx, params, shape)
        if len(r.shape) > len(p.shape):
            p = tf.expand_dims(p, axis=-1)
        res.append(r * p)
    return tf.add_n(res)

In [5]:
class TestCell(tf.contrib.rnn.RNNCell):
    def __init__(self, is_training):
        super().__init__(self)
        self.is_training = is_training
        
    def resize(self, tensor):
        return tf.image.resize_images(
            tensor,
            (config.TAR_H, config.TAR_W),
            method=tf.image.ResizeMethod.BICUBIC,
            align_corners=True)
    
    def __call__(self, frame_tensor, state):
        model_output, _ = mobilenet_v2.mobilenet_base(frame_tensor, output_stride=config.STRIDE)
        state = tf.reshape(state, RESULT_SHAPE)
        
        model_output = tf.concat([model_output, state], axis=-1)
        _, so_x_prev, so_y_prev, mo_x_prev, mo_y_prev = tf.split(state, DEPTH, axis=-1)
        
        hm_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1])
        so_x_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1]) + so_x_prev
        so_y_pred = slim.conv2d(model_output, config.NUM_KP, [1, 1]) + so_y_prev
        mo_x_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1]) + mo_x_prev
        mo_y_pred = slim.conv2d(model_output, config.NUM_EDGE, [1, 1]) + mo_y_prev
        
        mv_x_pred = tf.squeeze(slim.conv2d(model_output, 1, [1, 1]), axis=[-1])
        mv_y_pred = tf.squeeze(slim.conv2d(model_output, 1, [1, 1]), axis=[-1])
        
        b, y, x = np.mgrid[:config.BATCH_SIZE, :MD_H, :MD_W]
        mv_p = [b, y + mv_y_pred, x + mv_x_pred]
        
        hm_expect = scatter_bilinear(hm_pred, mv_p, hm_pred.shape)
        so_x_expect = scatter_bilinear(so_x_pred, mv_p, so_x_pred.shape)
        so_y_expect = scatter_bilinear(so_y_pred, mv_p, so_y_pred.shape)
        
        b, y, x, i = np.mgrid[:config.BATCH_SIZE, :MD_H, :MD_W, :config.NUM_EDGE]
        for _ in range(config.NUM_RECURRENT):
            mo_p = [b, y+mo_y_pred, x+mo_x_pred, i]
            mo_x_pred = gather_bilinear(so_x_pred, mo_p) + mo_x_pred
            mo_y_pred = gather_bilinear(so_y_pred, mo_p) + mo_y_pred
        
        mo_p = [b, y+mo_y_pred, x+mo_x_pred]
        mo_x_expect_in_cp = gather_bilinear(mv_x_pred, mo_p) + \
                            mo_x_pred - tf.expand_dims(mv_x_pred, axis=-1)
        mo_y_expect_in_cp = gather_bilinear(mv_y_pred, mo_p) + \
                            mo_y_pred - tf.expand_dims(mv_y_pred, axis=-1)
        mo_x_expect = scatter_bilinear(mo_x_expect_in_cp, mv_p, mo_x_pred.shape)
        mo_y_expect = scatter_bilinear(mo_y_expect_in_cp, mv_p, mo_y_pred.shape)
        
        next_state = tf.concat([hm_expect, so_x_expect, so_y_expect, mo_x_expect, mo_y_expect], axis=-1)
        next_state = tf.reshape(next_state, [config.BATCH_SIZE, RESULT_SIZE])
        output = tf.concat([hm_pred, \
                            so_x_pred*config.STRIDE, so_y_pred*config.STRIDE, \
                            mo_x_pred*config.STRIDE, mo_y_pred*config.STRIDE], axis=-1)
        output = self.resize(output)
        output = tf.reshape(output, [config.BATCH_SIZE, OUTPUT_SIZE])
        
        return output, next_state

    @property
    def state_size(self):
        return RESULT_SIZE

    @property
    def output_size(self):
        return OUTPUT_SIZE


In [6]:

test_cell = TestCell(is_training=True)
#with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
with tf.device('/cpu:0'):
    pred_sum, _ = tf.nn.dynamic_rnn(test_cell, tensors['image'], sequence_length=tensors['seq_len'], dtype=tf.float32)

In [7]:
TOTAL_SHAPE = (config.BATCH_SIZE, config.MAX_FRAME_SIZE, config.TAR_H, config.TAR_W, sum(DEPTH))
pred_sum = tf.reshape(pred_sum, TOTAL_SHAPE)
hm_out, so_x_out, so_y_out, mo_x_out, mo_y_out = tf.split(pred_sum, DEPTH, axis=-1)

In [8]:

saver = tf.train.Saver()
with tf.device('/cpu:0'):
    with tf.Session() as sess:
        checkpoint_path = 'logs/log_test/model.ckpt-12133'
        saver.restore(sess, checkpoint_path)
        res = sess.run([tensors['image'], hm_out])


INFO:tensorflow:Restoring parameters from logs/log_test/model.ckpt-12133


InternalError: Dst tensor is not initialized.
	 [[Node: rnn/transpose_1/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_610_rnn/transpose_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]